# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [1]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

     |████████████████████████████████| 471kB 4.7MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.4-cp36-cp36m-linux_x86_64.whl size=271976 sha256=385b1d1d0b4491730711a312fbd3640b1887a449f685a2820f2fde60cfdecd36
  Stored in directory: /root/.cache/pip/wheels/58/30/73/8858a1b6e5e2674e2ea85c9904949c06addcf6fd34d59b5ea6
Successfully built pytrec-eval
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=91148a8aad107c5a0ca19301a77f278712a70026e7d1d5f923191c1295d7dd03
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government.zip'

In [3]:
!unzip government.zip

Archive:  government.zip
   creating: government/
  inflating: government/topics-with-full-descriptions.txt  
  inflating: government/gov.topics   
  inflating: government/gov.qrels    
   creating: government/documents/
   creating: government/documents/61/
  inflating: government/documents/61/G00-61-2800209  
  inflating: government/documents/61/G00-61-1192048  
  inflating: government/documents/61/G00-61-1118212  
  inflating: government/documents/61/G00-61-0749882  
  inflating: government/documents/61/G00-61-2230501  
  inflating: government/documents/61/G00-61-0680698  
  inflating: government/documents/61/G00-61-0551387  
  inflating: government/documents/61/G00-61-2575433  
  inflating: government/documents/61/G00-61-0469713  
  inflating: government/documents/61/G00-61-0280746  
  inflating: government/documents/61/G00-61-2574316  
  inflating: government/documents/61/G00-61-3933997  
  inflating: government/documents/61/G00-61-3290635  
  inflating: government/documents/61/G0

In [0]:
# imports
# Put all your imports here
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
from whoosh import qparser#used in Q4
from whoosh import scoring#used in Q4
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import nltk
from nltk.stem import *
from whoosh.analysis import Filter

In [0]:
DATA_DIR = "government"
DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

MAP, mean average precision.

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

Because MAP method is macro-averaging, a general method to measure search engine performance. By using MAP method, it weighted queries equally, and can measure the information and returned documents in ranked result. Thus, I think MAP should be appropriate for measuring search system performance for government web sites and appropriate for individual topic. 

## Question 2

### Q2 (a): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# the final index in the variable INDEX_Q2
# query parser in QP_Q2
# searcher in SEARCHER_Q2
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

# first, define a Schema for the index
mySchema_question2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

# now, create the index at the path INDEX_DIR based on the new schema
myINDEX_Q2 = createIndex(mySchema_question2)


#The schema is the set of all possible fields in a document. 
#Each individual document might only use a subset of the available fields in the schema.

In [0]:
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1 % 1000 == 0):
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [8]:
# Build a list of files to index
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

# count files to index
print("number of files:", len(filesToIndex))


addFilesToIndex(myINDEX_Q2, filesToIndex)

number of files: 4078
done indexing.


In [9]:
# define a query parser for the field "file_content" in the index
myQP_Q2 = QueryParser("file_content", schema=myINDEX_Q2.schema)
mySEARCHER_Q2 = myINDEX_Q2.searcher()

# run a sample query for the phrase "item"
sampleQuery = myQP_Q2.parse("genealogy searches")###topic 16###
sampleQueryResults = mySEARCHER_Q2.search(sampleQuery, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-26-1048210 0 12.268872570654011
G00-59-3622783 1 5.1327224570039185


In [10]:
# print the topic file
with open(TOPIC_FILE, "r") as f:
    print(f.read())

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education



In [11]:
# print the qrels file
with open(QRELS_FILE, "r") as f:
    qrels = f.readlines()
    print("".join(qrels))

1 0 G00-00-0681214 0
1 0 G00-00-0945765 0
1 0 G00-00-1006224 1
1 0 G00-00-1591495 0
1 0 G00-00-2764912 0
1 0 G00-00-3253540 0
1 0 G00-00-3717374 0
1 0 G00-01-0270065 0
1 0 G00-01-0400712 0
1 0 G00-01-0682299 0
1 0 G00-01-2154945 0
1 0 G00-01-2689026 0
1 0 G00-01-2898660 0
1 0 G00-02-0146077 0
1 0 G00-02-0351712 0
1 0 G00-02-0510219 0
1 0 G00-02-0555602 0
1 0 G00-02-0901987 1
1 0 G00-02-1239993 0
1 0 G00-02-3981961 0
1 0 G00-02-4057099 0
1 0 G00-03-0366425 0
1 0 G00-03-0697220 0
1 0 G00-03-0931579 0
1 0 G00-03-1585062 0
1 0 G00-03-1898526 1
1 0 G00-04-0296440 0
1 0 G00-04-0767639 0
1 0 G00-04-0971232 0
1 0 G00-04-1046533 0
1 0 G00-04-1562575 0
1 0 G00-04-1864045 0
1 0 G00-04-3558844 0
1 0 G00-04-4166204 0
1 0 G00-05-0619345 0
1 0 G00-05-0623935 0
1 0 G00-05-1315394 0
1 0 G00-05-2231767 0
1 0 G00-05-2357004 0
1 0 G00-05-2948873 0
1 0 G00-05-2988698 0
1 0 G00-05-3051665 0
1 0 G00-05-3539237 0
1 0 G00-06-0872838 0
1 0 G00-06-2562505 0
1 0 G00-06-3965004 0
1 0 G00-06-4038523 0
1 0 G00-07-11

In [0]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)
        #qrel, {'map'}) # chose map only
    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [13]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQP_Q2, mySEARCHER_Q2) 


num_q                    1       1.0000
num_ret                  1       1.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [0]:
INDEX_Q2= myINDEX_Q2
QP_Q2 = myQP_Q2
SEARCHER_Q2 = mySEARCHER_Q2

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

From above output data, Whoosh system overall MAP is 0.1971.Recall is 7/33(num_rel_ret/num_rel) and percision is 7/151(num_rel_ret/num_ret). Therefore, we can see the results of MAP, recall, and percision, they are all at very low rate.

### Q2 (c): Provide answer to Q2(c) here [markdown cell]

Topic 18 and topic 24 did really well because all relevent information has been retreived, and their MAP scores are 1. And total number of relevent docs equal to total number of relevent docs retrieved.

Topic 1,2,6,7,9,16,28, their MAP scores are all zero, there are no relevent docs retreived(or very less) for these topics, some of them do not have a score returned for them, really bad.

## Question 3

In [0]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

In [16]:
printRelName(TOPIC_FILE, QRELS_FILE, QP_Q2, SEARCHER_Q2, "9")

---------------------------Topic_id and Topic_phrase----------------------------------
9 genealogy searches
---------------------------Return documents----------------------------------
9 Q0 G00-26-1048210 0 12.268873 test
9 Q0 G00-59-3622783 1 5.132722 test
---------------------------Relevant documents----------------------------------
9 0 G00-91-3181951 1


I checked some aspects that may affect the performance of search engine. The codes below are the criterias I checked.

In [17]:
# define a reader object on the index
myReader = INDEX_Q2.reader()
#how many terms do we have?
print(myReader.field_length("file_content"))
# list indexed terms for field "file_content"
[term for term in myReader.field_terms("file_content")][90000:90050]

2165181


['quantifies',
 'quantify',
 'quantifying',
 'quantitating',
 'quantitation',
 'quantitative',
 'quantitatively',
 'quantities',
 'quantity',
 'quantization',
 'quantum',
 'quarantine',
 'quark',
 'quark.phy.bnl.gov',
 'quarknet.fnal.gov',
 'quarks',
 'quarries',
 'quart',
 'quarter',
 'quarter1.gif',
 'quarter2.gif',
 'quarter3.gif',
 'quarter4.gif',
 'quarterback',
 'quarterly',
 'quarters',
 'quartet',
 'quarts',
 'quartz',
 'quasar',
 'quasar.nih.gov',
 'quasars',
 'quash',
 'quashed',
 'quasi',
 'quasiparticle',
 'quasiparticles',
 'que',
 'queja',
 'quejas',
 'quences',
 'quench',
 'quenched',
 'quenching',
 'queri.research.med.va.gov',
 'queriable',
 'queried',
 'queries',
 'query',
 'query.asp']

We can see the terms above, most of them near each other have same meaning but different roots.

In [18]:
print("# docs with 'genealogy'", myReader.doc_frequency("file_content", "genealogy"))
print("# docs with 'genealogical'", myReader.doc_frequency("file_content", "genealogical"))
print("# docs with 'Genealogic'", myReader.doc_frequency("file_content", "Genealogy"))
print("# docs with 'search'", myReader.doc_frequency("file_content", "search"))
print("# docs with 'searches'", myReader.doc_frequency("file_content", "searches"))
print("# docs with 'Searches'", myReader.doc_frequency("file_content", "Searches"))

# docs with 'genealogy' 9
# docs with 'genealogical' 4
# docs with 'Genealogic' 21
# docs with 'search' 581
# docs with 'searches' 50
# docs with 'Searches' 24


In [19]:
with open(DOCUMENTS_DIR+'/26/G00-26-1048210','r') as f:#open a false positive
  print(f.read())

http://unix2.nysed.gov/faq/

   New York State Library

                        Frequently Asked Questions (FAQ)

               Online Catalog, Web, Telnet, Scanned Documents
     * How do I Access the online catalog via Web browser? Telnet? Modem?
       Z39.50?
     * Where can I find help on searching Excelsior? Searching for a
       scanned document?
     * Why can't I get into WebCat? What does "session in use" mean?
     * Why am I getting a server error when I try to access a scanned
       document?
     * Why is the printer printing the same record (not printing records
       from multiple searches)?
     * After installing QuickTime, I cannot get any other TIFF viewer to
       work in Netscape. How can I fix this?

                           New York State Library
     * Who is eligible to borrow materials?
     * How does Interlibrary Loan work?
     * What newspapers does the Library have?
     * How do I access materials in Manuscripts and Special Collections?
     * W

In [20]:
with open(DOCUMENTS_DIR+'/91/G00-91-3181951','r') as f:#open fn
  print(f.read())

http://ferguson.library.ca.gov/html/genealogy.html


   State of California Website
                                      [clearpixel.gif]
   CSL Catalog
   Highlights
   Library of California
   Only Just Begun
   Connection - the CSL Newsletter
   Gold Rush
   Library Bond Act of 2000
   Linking Welfare Recipients to Jobs
   Proposition 218 After Two Years
   Trustee Toolkit
   LSTA
   Civil Liberties Grants
   Shortcuts
   Home
   CRB Reports
   California Libraries
   California Family Impact Seminar (CAFIS)
   Genealogy
   Library Laws
   General Info
   Mission Statement
   Overview
   Location/Hours
   Staff
   Jobs at the California State Library
   Search Site

   SELECTED GUIDE TO SOURCES FOR GENEALOGY IN THE CALIFORNIA HISTORY
   SECTION
   [clearpixel.gif] [clearpixel.gif] [clearpixel.gif] [clearpixel.gif]
   [clearpixel.gif] [clearpixel.gif] [clearpixel.gif] [clearpixel.gif]
   [clearpixel.gif]
   Gold Bar
     * INTRODUCTION
     * CALIFORNIA CENSUS MATERIALS
     * COMPI

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

Take quary 9: genealogy searches as example, there are two results were listed in the sample result output: 





*   G00-26-1048210 0 12.268872570654011;
*   G00-59-3622783 1 5.1327224570039185;



For document G00-26-1048210 and G00-59-3622783, they did not show in qrels file, shoud be a false positive. By looking at the content in the document "G00-26-1048210", which is just the FAQ of public, there is not much useful information related to "genealogy searches". It shows up just because it contains a word: Genealogical Research.

Then from qrels file, I taka G00-91-3181951 as example,which means that it is revelent, but it does not show in the sample result list, should be a false negative. By looking at the content of it, this is a guide to the most frequently used genealogical sources in the California Section and a list of private researchers. This document is actually talking about "genealogy" related topics but it didnt extracted out, the key words may shown in different or various forms.


---
From above printed result, we can tell that there are some aspects may affect Whoosh's performance, for example, the documents contain the queries but in different format: stop words appear too many, upper/lower case, and the words with same root(genealogical/genealogy), break phrases like "quasar.nih.gov" etc. 

Adjusting the words in the queries before search will help.
Thus, I think by using NLTK's stemmers and lemmatizers stemming method, LancasterStemmer(), LowercaseFilter, and StopFilter should be used to improve performance.

For the following analysis, I choose to use NLTK method.


### Q3 (b): Write your code below

In [21]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [23]:

myFilter = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | CustomFilter(LancasterStemmer().stem)
mySchemaQ3 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = myFilter))
#create index again
myINDEX_Q3 = createIndex(mySchemaQ3)

addFilesToIndex(myINDEX_Q3, filesToIndex)

done indexing.


In [0]:
myQP_Q3 = QueryParser("file_content", schema=myINDEX_Q3.schema)
mySEARCHER_Q3 = myINDEX_Q3.searcher()

In [25]:
# run a sample query for the phrase "item"
sampleQuery = myQP_Q3.parse("genealogy searches")
sampleQueryResults = mySEARCHER_Q3.search(sampleQuery, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-30-0221651 0 14.375037867758056
G00-79-2892445 1 13.424950588630212
G00-26-1048210 2 12.392029318651186
G00-55-0643570 3 11.497054143096872
G00-02-1372443 4 10.777541937361102
G00-08-1314254 5 10.777541937361102
G00-08-0900666 6 10.777541937361102
G00-59-0523165 7 10.405808918386139
G00-95-3755341 8 10.405808918386139
G00-88-2629440 9 10.405808918386139
G00-06-1975174 10 10.405808918386139
G00-95-3337324 11 10.353918860271968
G00-24-0016657 12 10.353918860271968
G00-01-2134408 13 10.1814202423583
G00-33-1729611 14 10.070068981217394
G00-01-2898660 15 9.879990942669014
G00-91-3181951 16 9.815246958879358
G00-43-3812747 17 9.743915969345494
G00-21-1529615 18 9.435812076690379
G00-67-1176122 19 9.147730089331123
G00-08-3780534 20 9.093147175005733
G00-49-2630728 21 8.601572598020624
G00-00-2016453 22 8.542644070469915
G00-59-3622783 23 7.592934911703263
G00-48-2464830 24 6.1386074695904185
G00-22-3171818 25 6.031816992720081
G00-21-2737022 26 4.565078287082096


In [26]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQP_Q3, mySEARCHER_Q3)

num_q                    1       1.0000
num_ret                  1       3.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [0]:
def printRelNameQ3(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

In [28]:
printRelNameQ3(TOPIC_FILE, QRELS_FILE, myQP_Q3, mySEARCHER_Q3, "9")

---------------------------Topic_id and Topic_phrase----------------------------------
9 genealogy searches
---------------------------Return documents----------------------------------
9 Q0 G00-30-0221651 0 14.375038 test
9 Q0 G00-79-2892445 1 13.424951 test
9 Q0 G00-26-1048210 2 12.392029 test
9 Q0 G00-55-0643570 3 11.497054 test
9 Q0 G00-02-1372443 4 10.777542 test
9 Q0 G00-08-1314254 5 10.777542 test
9 Q0 G00-08-0900666 6 10.777542 test
9 Q0 G00-59-0523165 7 10.405809 test
9 Q0 G00-95-3755341 8 10.405809 test
9 Q0 G00-88-2629440 9 10.405809 test
9 Q0 G00-06-1975174 10 10.405809 test
9 Q0 G00-95-3337324 11 10.353919 test
9 Q0 G00-24-0016657 12 10.353919 test
9 Q0 G00-01-2134408 13 10.181420 test
9 Q0 G00-33-1729611 14 10.070069 test
9 Q0 G00-01-2898660 15 9.879991 test
9 Q0 G00-91-3181951 16 9.815247 test
9 Q0 G00-43-3812747 17 9.743916 test
9 Q0 G00-21-1529615 18 9.435812 test
9 Q0 G00-67-1176122 19 9.147730 test
9 Q0 G00-08-3780534 20 9.093147 test
9 Q0 G00-49-2630728 21 8.601573 

In [0]:
INDEX_Q3 = myINDEX_Q3
QP_Q3 = myQP_Q3
SEARCHER_Q3 = mySEARCHER_Q3

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

I used NLTK's stemmers and lemmatizers stemming method, added RegexTokenizer() to split the sentance to words, LowercaseFilter to lower the case of words, then, used StopFilter to get rid off the non relevent common stop words.


---



Overall, the MAP score of 0.3456, which is now higher then the previous score (0.1971). And gm_ap increased from 0.0015 to 0.0187. Thus, we can tell by doing the improvement methods mentioned above, we can actually improve the performace. 


---



Also, the false negative example (G00-91-3181951) are taken care of already. It is shown in the sample result now. 
False positive problems are still there. Previous we returned only 2 files, but now we returned 27 files and none of them are in qurel file. 

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

Yes, it did improve the performance.

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

Yes, topic 28 perviously only had 0 MAP score, but now it has 0.2262, thus, it got better. 
However, for example, topic 26 dropped from 0.1111 to 0.0771, it got worse.
Also, there are still false positives in the result and the number of false positives are more than before.

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

I think the overall performance is improved because we get better MAP score, as well as better gm_map score. Then, there are more document retreieved than before since those filters helped us finding more relevent results. In addition, the false negative issue in system was fixed by doing so. Which in someway, increased the false positive in system but overall performance is improved.

## Question 4

In [30]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4
Q4Analyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()| CustomFilter(LancasterStemmer().stem)| CustomFilter(WordNetLemmatizer().lemmatize, 'v')| CustomFilter(WordNetLemmatizer().lemmatize)

mySchemaQ4 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = Q4Analyzer))

# create the index based on the new schema
myINDEX_Q4 = createIndex(mySchemaQ4)

addFilesToIndex(myINDEX_Q4, filesToIndex)

done indexing.


**Selection of factory, B, K1 values:**
step 1: OR group only. I add OR query method into my queryParser function to allow the search engine to search term by using OR query instead of AND. 

The factory number is between 0 and 1, so I tried different number for factory number and found out that 0.8 can provide best performance.

In [0]:
QP_Q4factory = QueryParser("file_content", schema=myINDEX_Q4.schema, group=qparser.OrGroup.factory(0.8))
SEARCHER_Q4factory = myINDEX_Q4.searcher()

In [32]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q4factory, SEARCHER_Q4factory)

num_q                    1       1.0000
num_ret                  1       470.0000
num_rel                  1       5.0000
num_rel_ret              1       5.0000
map                      1       0.0627
gm_map                   1       -2.7699
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0526
iprec_at_recall_0.00     1       0.0938
iprec_at_recall_0.10     1       0.0938
iprec_at_recall_0.20     1       0.0938
iprec_at_recall_0.30     1       0.0938
iprec_at_recall_0.40     1       0.0938
iprec_at_recall_0.50     1       0.0938
iprec_at_recall_0.60     1       0.0938
iprec_at_recall_0.70     1       0.0435
iprec_at_recall_0.80     1       0.0435
iprec_at_recall_0.90     1       0.0435
iprec_at_recall_1.00     1       0.0435
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.06

Step 2: Right now the map = 0.3892. Because we cannot pass the base line 0.41 by just adding OR group, another way is to use scoring BM25F function into my searcher.

In [0]:
myQP_Q4 = QueryParser("file_content", schema=myINDEX_Q4.schema, group=qparser.OrGroup.factory(0.8))
mySEARCHER_Q4 = myINDEX_Q4.searcher(weighting=scoring.BM25F(B=0.5,K1=5))#page 163
#https://buildmedia.readthedocs.org/media/pdf/whoosh/latest/whoosh.pdf
#b0.50 k5    map0.4106
#the variable k1 is a smoothing parameter for adjusting term frequencies saturation. 
#The variable b controls how much effect field-length normalization should have. 


#K
#This parameter controls how quickly an increase in term frequency results in term-frequency saturation. 
#The default value is 1.2. 
#Lower values result in quicker saturation, and higher values in slower saturation.


#B
#This parameter controls how much effect field-length normalization should have. 
#A value of 0.0 disables normalization completely, and a value of 1.0 normalizes fully. 
#The default is 0.75.


In [34]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQP_Q4, mySEARCHER_Q4)

num_q                    1       1.0000
num_ret                  1       470.0000
num_rel                  1       5.0000
num_rel_ret              1       5.0000
map                      1       0.0682
gm_map                   1       -2.6858
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0556
iprec_at_recall_0.00     1       0.0909
iprec_at_recall_0.10     1       0.0909
iprec_at_recall_0.20     1       0.0909
iprec_at_recall_0.30     1       0.0909
iprec_at_recall_0.40     1       0.0909
iprec_at_recall_0.50     1       0.0909
iprec_at_recall_0.60     1       0.0909
iprec_at_recall_0.70     1       0.0625
iprec_at_recall_0.80     1       0.0625
iprec_at_recall_0.90     1       0.0485
iprec_at_recall_1.00     1       0.0485
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.06

In [35]:
printRelName(TOPIC_FILE, QRELS_FILE, myQP_Q4, mySEARCHER_Q4, "9")

---------------------------Topic_id and Topic_phrase----------------------------------
9 genealogy searches
---------------------------Return documents----------------------------------
9 Q0 G00-30-0221651 0 13.704827 test
9 Q0 G00-79-2892445 1 13.065959 test
9 Q0 G00-26-1048210 2 9.465239 test
9 Q0 G00-91-3181951 3 8.414106 test
9 Q0 G00-55-0643570 4 7.484404 test
9 Q0 G00-02-1372443 5 7.098417 test
9 Q0 G00-08-1314254 6 7.098417 test
9 Q0 G00-08-0900666 7 7.098417 test
9 Q0 G00-43-3812747 8 6.449216 test
9 Q0 G00-95-3337324 9 6.324979 test
9 Q0 G00-24-0016657 10 6.324979 test
9 Q0 G00-01-2134408 11 6.217790 test
9 Q0 G00-48-2464830 12 5.933189 test
9 Q0 G00-28-3598417 13 5.728867 test
9 Q0 G00-01-2898660 14 5.706292 test
9 Q0 G00-59-0523165 15 5.584208 test
9 Q0 G00-95-3755341 16 5.584208 test
9 Q0 G00-88-2629440 17 5.584208 test
9 Q0 G00-06-1975174 18 5.584208 test
9 Q0 G00-59-3622783 19 5.522089 test
9 Q0 G00-33-1729611 20 5.411123 test
9 Q0 G00-08-3780534 21 5.250793 test
9 Q0 G00

### Please answer the following questions here

(a) A clear list of all final modifications made.  


*   Adding | StemFilter()| CustomFilter(LancasterStemmer().stem)| CustomFilter(WordNetLemmatizer().lemmatize, 'v')| CustomFilter(WordNetLemmatizer().lemmatize)

*   Using the factory() class method of Orgroup:
QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema, group=qparser.OrGroup.factory(0.8))

*   Using whoosh scoring BM25F function: SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.BM25F(B=0.5,K=5))




(b)  Why each modification was made – how did it help? 

*   By adding more filters to improve the performance.

*   I used 'OR' query in this question, it lets me specify that documents that contain more of the query terms score higher. By using OR query, we can get much more relevent documents return. So we can increase map score by doing so. Because the factory number only between 0 and 1, it is easilier by using the trial and error method to find the best factory number, i found that the factory=0.8 can provide the best result.

*   Also, I used whoosh scoring BM25F function, I have tried different B,K combinations and find out B = 0.5, k=5, can result map 0.4107. Since B and K combinations can be varied so differently, it is hard to find the best B and K values that can provided best performance. But there is certainly exist a better B and K combination which can provide better result. 






(c)  The  final  MAP  performance  that  these  modifications  attained.


{**MAP = 0.4107**; Improved and above the 0.41 line.}

After running the pyterc, the false positive cases getting worse, the system return so many false nagatives. But false nagative case was been taken care of. And the overall MAP and gm_map scores are better than before.

In [0]:
INDEX_Q4 = myINDEX_Q4 # Replace None with your index for Q4
QP_Q4 = myQP_Q4# Replace None with your query parser for Q4
SEARCHER_Q4 = mySEARCHER_Q4# Replace None with your searcher for Q4

## Validation

In [0]:
# Run the following cells to make sure your code returns the correct value types

In [0]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [39]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [40]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation (Graduate Students)

In [41]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
